In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from numpy import meshgrid

from geopy.geocoders import Nominatim
from tqdm import tqdm
import indices_no_leap as ii

# Figure 3

In [ ]:
map_bounds=[65, 40, 100, 5]

In [ ]:
from tqdm import tqdm
def check_for_nans_infs_zeros(data_sts):
    print('Nans:',np.sum(np.isnan(data_sts)))
    print('Infs:' ,np.sum(np.isinf(data_sts)))
    print('Zeros:',np.sum((data_sts==0)))
    
    
def spatial_ind_single(func,data_all,mask):
    N=data_all.shape[0]
    N1=data_all.shape[1]
    N2=data_all.shape[2]
    Nyears  =  int(N/365 )

    calc_index=np.zeros((N1,N2))
    for i in range(N1):
      for j in range(N2):
        if mask[i,j] == 0:
            calc_index[i,j]=np.nanmean(func(data_all[:,i,j],Nyears))
    return calc_index



def spatial_ind_single_var(func,data_all,mask):
    N=data_all.shape[0]
    N1=data_all.shape[1]
    N2=data_all.shape[2]
    Nyears  =  int(N/365 )
    
    calc_index=np.zeros((N1,N2))
    for i in range(N1):
      for j in range(N2):
            if mask[i,j] == 0:
                calc_index[i,j]=np.nanvar(func(data_all[:,i,j],Nyears))
    return calc_index



## Import data

In [ ]:
output_figname = ''
path_out_IMD = '/home/vikram/BCSD_data/India_P_BCSDd'+output_figname

In [ ]:
gcm_list = ['CanESM2']
index_list = ['CDD','Rx1day','R20mm','R99p']
N_random = 100


In [ ]:
data_all_indices = {}


gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_list[0]+'.npy',allow_pickle=True)
Nyears_train = gcm_meta.item()['Nyears_train']
N_future_start = 2045-2006

st=gcm_meta.item()['st']
en=gcm_meta.item()['en']

lat_obs=gcm_meta.item()['lat_obs']
lon_obs=gcm_meta.item()['lon_obs']
mask=gcm_meta.item()['mask']


N1=lat_obs.shape[0]
N2=lon_obs.shape[0]

imd_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_obs_imd_pred.npy'))[365*Nyears_train:]
for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
    data_all_indices[ind_name] = [] 
    data_all_indices[ind_name].append(spatial_ind_single(func,imd_test,mask))

data_all_indices["Var"] = [] 
data_all_indices["Var"].append(spatial_ind_single_var(ii.sum_annualy,imd_test,mask))


In [ ]:
mask_nan = np.array(mask , dtype =np.float16)
mask_nan[mask == True] = np.nan
mask_nan[mask == False] = 1

In [ ]:
for gcm_name in tqdm(gcm_list):
    gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_name+'.npy',allow_pickle=True)
    lat_gcm_01=gcm_meta.item()['lat_gcm']
    lon_gcm_01=gcm_meta.item()['lon_gcm']
    gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_gcm_'+gcm_name+'_imd_pred.npy'))[365*Nyears_train:]
    data_gf_test=np.zeros(imd_test.shape)
    for i in range(gcm_test.shape[0]):
        data_gf_test[i,:,:]=uu.regrid(gcm_test[i,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
        
    for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
        data_all_indices[ind_name].append(spatial_ind_single(func,data_gf_test,mask))
    data_all_indices["Var"].append(spatial_ind_single_var(ii.sum_annualy,data_gf_test,mask))
    
del data_gf_test
    
for gcm_name in tqdm(gcm_list):
    out_id='pre_'+gcm_name 
    bcsd_gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/detrended_BCSD_outputs_historical'+out_id+'.npy'))[365*Nyears_train:]
    # data_all_indices["Mean"].append(spatial_ind_single(ii.sum_annualy,bcsd_gcm_test,mask))
    
    for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
        data_all_indices[ind_name].append(spatial_ind_single(func,bcsd_gcm_test,mask))
    data_all_indices["Var"].append(spatial_ind_single_var(ii.sum_annualy,bcsd_gcm_test,mask))

del bcsd_gcm_test    
    
for gcm_name in tqdm(gcm_list):
    gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_name+'.npy',allow_pickle=True)
    lat_gcm_01=gcm_meta.item()['lat_gcm']
    lon_gcm_01=gcm_meta.item()['lon_gcm']

    gcm_future=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_gcm_'+gcm_name+'_future.npy'))[365*N_future_start:]
    data_gf_future=np.zeros([gcm_future.shape[0],imd_test.shape[1],imd_test.shape[2]])
    for i in range(data_gf_future.shape[0]):
        data_gf_future[i,:,:]=uu.regrid(gcm_future[i,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
    # data_all_indices["Mean"].append(spatial_ind_single(ii.sum_annualy,data_gf_future,mask))
    for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
        data_all_indices[ind_name].append(spatial_ind_single(func,data_gf_future,mask))
    data_all_indices["Var"].append(spatial_ind_single_var(ii.sum_annualy,data_gf_future,mask))
        
del  data_gf_future,gcm_future


for gcm_name in tqdm(gcm_list):
    bcsd_gcm_future2=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/detrended_BCSD_outputs_'+out_id+'.npy'))[365*N_future_start:]
    # data_all_indices["Mean"].append(spatial_ind_single(ii.sum_annualy,bcsd_gcm_future2,mask))
    for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
        data_all_indices[ind_name].append(spatial_ind_single(func,bcsd_gcm_future2,mask))
    data_all_indices["Var"].append(spatial_ind_single_var(ii.sum_annualy,bcsd_gcm_future2,mask))
del  bcsd_gcm_future2
    


    
    
    

In [ ]:
data_all_indices_random={}


for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
    data_all_indices_random[ind_name] = [] 
data_all_indices_random["Var"] = [] 

for j in tqdm(range(1)):
    out_id='pre_rnd_new_'+str(j)  
    bcsd_rnd_test=uu.correct_nan_inf_neg((np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy')))[365*Nyears_train:]
    for ind_name, func in zip(['Mean','CDD','Rx1day','R20mm','R99p'],[ii.sum_annualy,ii.CDD_annualy,ii.Rx1day_annualy,ii.R20mm_annualy,ii.R99p_annualy]):
        data_all_indices_random[ind_name].append(spatial_ind_single(func,bcsd_rnd_test,mask))
    data_all_indices_random["Var"].append(spatial_ind_single_var(ii.sum_annualy,bcsd_rnd_test,mask))


In [ ]:
period_list  =["ESM:1950-2005",
              "BCSD:1950-2005",
              "ESM:2045-2099",
              "BCSD:2045-2099",]

ind_name ="Mean"
ylims  =  [ 0 , 2000]
units = "mm"
cmap ='BrBG'

   
    

N_esms =len(gcm_list)

plt.figure(figsize  =  (10,6))

ax = plt.subplot(2,4,1)
plt.pcolor(lon_obs,lat_obs,data_all_indices[ind_name][0]*mask_nan,cmap = cmap,
          vmax= ylims[1],vmin= ylims[0])
plt.colorbar()

plt.text(0.995, 0.995, "Observations",  horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
   fontsize=18,color='k')

for piter in range(4):
    dataiter = piter + 1 
    subiter = (piter% 1 ) * 4    + (piter//1) + 1
    
    # print(subiter,dataiter)
    ax =plt.subplot(2,4,4+ subiter)
    plt.pcolor(lon_obs,lat_obs,data_all_indices[ind_name][dataiter]*mask_nan,cmap = cmap,
              vmax= ylims[1],vmin=ylims[0]
              )
    plt.colorbar(label = units)
#     plt.text(0.995, 0.995, gcm_list[(piter% 9 )], horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
#        fontsize=18,color='k')

#     plt.text(0.995, 0.05, period_list[piter%4], horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight='bold',
#        fontsize=18,color='k')

plt.tight_layout()



# plt.savefig("/home/vikram/Dropbox/BCSD Work/03_Figures/plots_Oct10/" + ind_name+ "_esms.png",dpi = 150,facecolor = 'w')
            
# plt.figure(figsize  =  (15,6.5))

# data_to_plot =   [
#     data_all_indices_random[ind_name][0],
#     np.percentile(data_all_indices_random[ind_name],25,axis = 0),
#     np.percentile(data_all_indices_random[ind_name],50,axis = 0),
#     np.percentile(data_all_indices_random[ind_name],75,axis = 0),

#     data_all_indices_random[ind_name][5],
#     np.mean(data_all_indices_random[ind_name],axis = 0),
#     np.percentile(data_all_indices_random[ind_name],75,axis = 0)-np.percentile(data_all_indices_random[ind_name],25,axis = 0),
#     np.var(data_all_indices_random[ind_name],axis = 0),

# ]

# data_to_plot_desc = ["Random 1", "25 %ile", "Median", "75 %ile",
#                       "Random 2","Mean","IQR","Var"]

# for piter in range(8):
#     ax =plt.subplot(2,4,piter+1)
    
#     if piter == 6 :
#         plt.pcolor(lon_obs,lat_obs,data_to_plot[piter]*mask_nan,cmap =cmap,
#                   vmax= ylims[1]/20,vmin=0
#                   )
#     elif  piter == 7:
#         plt.pcolor(lon_obs,lat_obs,data_to_plot[piter]*mask_nan,cmap =cmap,
#                   vmax= ylims[1]/5,vmin=0
#                   )
#     else:
#         plt.pcolor(lon_obs,lat_obs,data_to_plot[piter]*mask_nan,cmap = cmap,
#           vmax= ylims[1],vmin=ylims[0]
#                   )
        
#     plt.colorbar(label = units)
#     plt.text(0.995, 0.995, data_to_plot_desc[piter], horizontalalignment='right',verticalalignment='top', transform=ax.transAxes,fontweight='bold',
#        fontsize=18,color='k')

#     plt.text(0.995, 0.05, "BCSD:1945-2005", horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight='bold',
#        fontsize=18,color='k')
# plt.tight_layout()

# plt.savefig("/home/vikram/Dropbox/BCSD Work/03_Figures/plots_Oct10/" + ind_name+ "_random.png",dpi = 150,facecolor = 'w')